In [3]:
import pandas as pd
import re, string, unicodedata, nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

df = pd.read_csv('../../data/bases_disciplina_2/tweets_classificados.csv', encoding='utf-8')
df.head()

,id,data_tweet,texto,sentimento
0,0,Sun Jan 08 01:22:05 +0000 2017,���⛪ @ Catedral de Santo Antônio - Governador ...,Neutro
1,1,Sun Jan 08 01:49:01 +0000 2017,"� @ Governador Valadares, Minas Gerais https:/...",Neutro
2,2,Sun Jan 08 01:01:46 +0000 2017,"�� @ Governador Valadares, Minas Gerais https:...",Neutro
3,3,Wed Jan 04 21:43:51 +0000 2017,��� https://t.co/BnDsO34qK0,Neutro
4,4,Mon Jan 09 15:08:21 +0000 2017,��� PSOL vai questionar aumento de vereadores ...,Negativo


# ToDo 1

Altere as funções de tratamento de texto apresentadas em sala para que elas façam a remoção de links também.

Crie uma nova coluna chamada texto_tratado que conterá o resultado da aplicação das funções.

## Funções existentes

In [40]:
# Remove acentos
def normalize_accents(text):
    return unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')

# Remove pontuação
def remove_punctuaction(text):
    punctiations = string.punctuation
    table = str.maketrans({key: " " for key in punctiations})
    text = text.translate(table)
    return text

# Remove URLs
def remove_links(text):
    return re.sub(r"\S*https?:\S*", "", text)

# Converte para minusculua, aplica funções de normalização e retira espaços em branco adicionais
def normalize_str(text):
    text = text.lower()
    text = remove_links(text)
    text = remove_punctuaction(text)
    text = normalize_accents(text)
    text = re.sub(re.compile(r" +")," ", text)
    return ' '.join([w for w in text.split()])

# Função completa de tokenização com exclusão de stopwords e verificação de dtype
def tokenizer(text):
    stop_words = nltk.corpus.stopwords.words('english')

    if isinstance(text, str):
        text = normalize_str(text)
        text = ''.join([w for w in text if not w.isdigit()])
        text = word_tokenize(text)
        text = [x for x in text if x not in stop_words]
        text = [y for y in text if len(y) >= 2]
        return ' '.join([t for t in text])
    else:
        print('Not a string')

In [43]:
tokenizer('Meu link é asdasd(https://twitterr34321)sadasd')

'meu link'

# ToDo 2

Ao fazer a remoção de links, percebemos que algumas linhas da coluna texto_tratado possuem valores faltantes. Entretanto, o Python trata eles como ''(str) e nao como Null. Assim, um simples dropna nao resolve o problema.

Encontre uma forma de remover tais elementos. Dica: use o índice das linhas cujos elementos da coluna texto_tratado seja nulo.

In [44]:
df['texto_norm'] = df['texto'].apply(tokenizer)

In [49]:
df.loc[df['texto_norm'] == '', 'texto_norm'] = None

df.isna().sum()

id             0
data_tweet     0
texto          0
sentimento     0
texto_norm    12
dtype: int64

In [50]:
df.dropna(subset=['texto_norm'], inplace=True)

In [51]:
print(df.shape)

(5763, 5)
